In [ ]:
#1.Múlti select Dropdowns
#-------------------------------------------------
from dash import Dash, dcc, html
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import pandas as pd
import plotly.express as px

education = pd.read_csv("Data/states_all.csv", usecols=["STATE", "YEAR", "TOTAL_EXPENDITURE"])

app = Dash(__name__)

app.layout = html.Div([
   dcc.Dropdown(
        id="filter",
        options=education["STATE"].unique(),
        value=["CALIFORNIA","ALASKA"],
        multi=True
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),Input("filter", "value"))
def plot_line(state):
    fig = px.line(
        (education
         .query("STATE in @state and 1992 < YEAR < 2016")
         .reset_index()
        ), 
        x="YEAR", 
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title = f"Gasto a través del tiempo, Estados Seleccionados",
    ) 
    return fig


if __name__=="__main__":
    app.run()

In [ ]:
#2.Checklists_1
#-------------------------------------------------

education = pd.read_csv("Data/states_all.csv", usecols=["STATE", "YEAR", "TOTAL_EXPENDITURE", "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AVG_READING_4_SCORE","AVG_READING_8_SCORE"])

app = Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="scores",
        options=[
            "AVG_MATH_4_SCORE",
            "AVG_MATH_8_SCORE",
            "AVG_READING_4_SCORE",
            "AVG_READING_8_SCORE"
        ],
        value=["AVG_MATH_4_SCORE"]
    ),
    dcc.Graph(id="graph")    
])

@app.callback(
    Output("graph", "figure"),
    Input("scores", "value")
)
def plot_hist(scores):
    fig = px.histogram(
        education,
        x=scores,
        #barmode="overlay",
        title="Distribución de Resultados"
    ).update_traces(opacity=.6)
    return fig

app.run()

In [ ]:
#3.Checklists_2
#-------------------------------------------------

app = Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="filter",
        options= [{"label": state.title(), "value": state} for state in education["STATE"].unique()], #muchas opciones!
#        options = ["CALIFORNIA", "TEXAS", "FLORIDA", "OREGON"],
        value=["CALIFORNIA", "TEXAS"]
    ),
    dcc.Graph(id="graph")    
])



@app.callback(
    Output("graph", "figure"),
    Input("filter", "value")
)

def exp_by_state(state):
    fig = px.line(
        (education
         .query("STATE in @state and 1992 < YEAR < 2016 ")
         .groupby(["YEAR", "STATE"])
         .sum()
         .reset_index()
        ),
        x="YEAR",
        y="TOTAL_EXPENDITURE",
        color="STATE",
        title="Gasto a través del tiempo, Estados Seleccionados"
    )
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#4.Radio Button_1
#-------------------------------------------------

app = Dash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id="OLS Toggle",
        options=["Trendline On", "Trendline Off"],
        value="Trendline Off"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("OLS Toggle", "value"),
)
def plot_scatter(ols):
    fig = px.scatter(
        education, 
        x="TOTAL_EXPENDITURE", 
        y="AVG_MATH_8_SCORE",
        trendline=None if ols == "Trendline Off" else "ols",
        title = "Relación entre Gasto y Resultados de Evaluación",
        )
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#5.Radio Button_2
#-------------------------------------------------

app = Dash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id="aggfunc",
        options=["mean", "sum", "median"],
        value="mean"
    ),
    dcc.Graph(id="graph")
])

@app.callback(
    Output("graph", "figure"),
    Input("aggfunc", "value")
)
def plot_rev_metric(metric):
    fig = px.bar(
        (education
        .query("STATE in ['CALIFORNIA', 'OREGON', 'WASHINGTON', 'IDAHO'] and 1992 < YEAR < 2016")
        .groupby(["STATE"])
        .agg({"TOTAL_EXPENDITURE": metric})
        .reset_index()
        ),
        x="STATE",
        y="TOTAL_EXPENDITURE",
        text_auto=True,
        title = f"Gasto por Estado: {metric.title()}"
    )
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#6.Slider simple
#-------------------------------------------------
import numpy as np

education = pd.read_csv("Data/states_all.csv").assign(
    expenditure_per_student = lambda x: x["TOTAL_EXPENDITURE"] / x["GRADES_ALL_G"]
)

app = Dash(__name__)

app.layout = html.Div([
    dcc.Slider(id="spend", min=0, max=26, step=2, value=12),
    dcc.Graph(id="graph"),
])

@app.callback(Output('graph', 'figure'), Input("spend", "value"))
def bar_chart(spending):
    fig = px.bar(
        (education
         .query("expenditure_per_student > @spending")
         .groupby("YEAR", as_index=False)
         .count()),
        x="YEAR",
        y="expenditure_per_student",
        title=f"Estados que gastaron más de ${spending}K por estudiante"      
    )
    fig.update_yaxes(title="Cuenta", range=[0, 70])
    fig.update_xaxes(range=[1991, 2017])
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#7.Slider Rango
#-------------------------------------------------

app = Dash(__name__)

app.layout = html.Div([
    dcc.RangeSlider(id="spend", min=0, max=30, step=2, value=[10, 14]),
    dcc.Graph(id="graph"),
])

@app.callback(Output('graph', 'figure'), Input("spend", "value"))
def bar_chart(spending):
    fig = px.bar(
        (education
         .query("@spending[0] <= expenditure_per_student <= @spending[1]")
         .groupby("YEAR", as_index=False)
         .count()),
        x="YEAR",
        y="expenditure_per_student",
        title=f"Estados que gastaron entre ${spending[0]}-{spending[1]}K por estudiante"      
    )
    fig.update_yaxes(title="Cuenta", range=[0, 50])
    fig.update_xaxes(range=[1991, 2017])
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#8.Dataframe a trabajar para DatePickers
#-------------------------------------------------
from datetime import datetime
from datetime import date

collisions = (
    pd.read_csv("Data/NYC_Collisions.csv")
    .astype({"ACCIDENT_DATE": "datetime64[ns]"})
    .loc[:,["ACCIDENT_DATE", "BOROUGH", "COLLISION_ID"]]
    .groupby(["ACCIDENT_DATE", "BOROUGH"], as_index=False)
    .count()
    .rename(columns={
        "COLLISION_ID":"COLLISIONS",
        "ACCIDENT_DATE":"DATE"})  
)

collisions.head()

In [ ]:
#9.DatePicker Single
#-------------------------------------------------

app = Dash(__name__)

app.layout = html.Div([
    dcc.DatePickerSingle(
        id="date picker",
        min_date_allowed=collisions["DATE"].min(),
        max_date_allowed=collisions["DATE"].max(),
        date=collisions["DATE"].min(),
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output("graph", "figure"),Input("date picker", "date"))
def plot_collisions_bar(date):
    fig = px.bar(
        (collisions
         .loc[collisions["DATE"].eq(date)]
         .groupby("BOROUGH", as_index=False)
         .sum(numeric_only=True)
        ),
        x="COLLISIONS",
        y="BOROUGH",
        title=f"Accidentes de Tráfico en NYC el día {date[:10]}"
    )
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#10.DatePicker Range
#-------------------------------------------------
app = Dash(__name__)

app.layout = html.Div([
    dcc.DatePickerRange(
        id="date picker",
        min_date_allowed=collisions["DATE"].min(),
        max_date_allowed=collisions["DATE"].max(),
        initial_visible_month=collisions["DATE"].max(),
        start_date=collisions["DATE"].min(),
        end_date=date(2022,1,1)
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output("graph", "figure"), 
              [Input("date picker", "start_date"),
               Input("date picker", "end_date"),])
def plot_collisions_bar(start_date, end_date):
    fig = px.bar(
        (collisions
         .loc[collisions["DATE"].between(start_date, end_date)]
         .groupby("BOROUGH", as_index=False)
         .sum(numeric_only=True)
        ),
        x="COLLISIONS",
        y="BOROUGH",
        title=f"Accidentes de Tráfico en NYC entre {start_date[:10]} y {end_date[:10]}"
    )
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#11.Múltiples input callbacks
#-------------------------------------------------
app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="X Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="expenditure_per_student"
),
    dcc.Dropdown(
        id="Y Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="TOTAL_EXPENDITURE"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("X Column Picker", "value"),
    Input("Y Column Picker", "value")
)

def plot_rev_scatter(x, y):
    if not x:
        raise PreventUpdate
    if not y:
        raise PreventUpdate
    fig = px.scatter(
        education, 
        x=x, 
        y=y,
        trendline="ols"
        )
    return fig

if __name__ == "__main__":
    app.run()

In [ ]:
#12.Múltiples output callbacks
#-------------------------------------------------

app = Dash(__name__)

app.layout = html.Div([
    html.H2(id="header"),
    dcc.Dropdown(
        id="X Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="expenditure_per_student"
),
    dcc.Dropdown(
        id="Y Column Picker",
        options=list(education.select_dtypes(include='number').columns),
        value="TOTAL_EXPENDITURE"
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('header', 'children'),
    Output('graph', 'figure'),
    Input("X Column Picker", "value"),
    Input("Y Column Picker", "value")
)

def plot_rev_scatter(x, y):
    if not x:
        raise PreventUpdate
    if not y:
        raise PreventUpdate
    fig = px.scatter(
        education, 
        x=x, 
        y=y,
        trendline="ols",
    )
    
    header = f"{x.title().replace('_', ' ')} Vs. {y.title().replace('_', ' ')}"
    return header, fig

if __name__ == "__main__":
    app.run()